In [20]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout


In [13]:
# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)


In [22]:
# Dataset parameters
dataset_path = 'dataset2'  # Replace with your dataset path
img_height = 224
img_width = 224
batch_size = 32

In [23]:
def prepare_data(dataset_path):
    # Create base directories
    base_dir = os.path.dirname(dataset_path)
    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'validation')
    test_dir = os.path.join(base_dir, 'test')
    
    # Remove existing directories and recreate
    for dir_path in [train_dir, val_dir, test_dir]:
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
        os.makedirs(dir_path)
    
    # Get class names (folders in the dataset path)
    classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    
    # Split data for each class
    for cls in classes:
        cls_path = os.path.join(dataset_path, cls)
        
        # Get all image files
        files = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        
        # Shuffle files
        np.random.shuffle(files)
        
        # Calculate split indices
        total_files = len(files)
        train_split = int(0.7 * total_files)
        val_split = train_split + int(0.2 * total_files)
        
        # Create class-specific directories
        train_cls_dir = os.path.join(train_dir, cls)
        val_cls_dir = os.path.join(val_dir, cls)
        test_cls_dir = os.path.join(test_dir, cls)
        
        os.makedirs(train_cls_dir, exist_ok=True)
        os.makedirs(val_cls_dir, exist_ok=True)
        os.makedirs(test_cls_dir, exist_ok=True)
        
        # Copy files to respective directories
        for i, file in enumerate(files):
            src = os.path.join(cls_path, file)
            if i < train_split:
                dst = os.path.join(train_cls_dir, file)
            elif i < val_split:
                dst = os.path.join(val_cls_dir, file)
            else:
                dst = os.path.join(test_cls_dir, file)
            
            # Use copy instead of symlink
            shutil.copy2(src, dst)
    
    return train_dir, val_dir, test_dir

In [24]:
# Data augmentation and generators
def create_data_generators(train_dir, val_dir, test_dir):
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    # Validation and test data generators (only rescaling)
    val_test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    return train_generator, val_generator, test_generator

In [25]:
def build_model(num_classes):
    # Load pre-trained DenseNet121
    base_model = DenseNet121(
        weights='imagenet', 
        include_top=False, 
        input_shape=(img_height, img_width, 3)
    )
    
    # More conservative fine-tuning
    # Only unfreeze the last few blocks
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    for layer in base_model.layers[-20:]:
        layer.trainable = True
    
    # Add regularization
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    
    return model

# When compiling
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Main training function with fine-tuning specific configurations
def train_model():
    # Prepare data
    train_dir, val_dir, test_dir = prepare_data(dataset_path)
    
    # Create data generators
    train_generator, val_generator, test_generator = create_data_generators(
        train_dir, val_dir, test_dir
    )
    
    # Get number of classes
    num_classes = len(train_generator.class_indices)
    print(f"Number of classes detected: {num_classes}")
    print(f"Class indices: {train_generator.class_indices}")
    
    # Build model
    model = build_model(num_classes)
    
    # Compile model with lower learning rate for fine-tuning
    model.compile(
        optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    checkpoint = ModelCheckpoint(
        'best_densenet_finetuned_model.keras', 
        save_best_only=True, 
        monitor='val_accuracy'
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=10, 
        restore_best_weights=True
    )
    
    # Learning rate reduction callback
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2,  # Reduce learning rate by 80%
        patience=5,  # Wait 5 epochs before reducing
        min_lr=1e-6  # Minimum learning rate
    )
    
    # Train model
    history = model.fit(
        train_generator,
        epochs=5,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stopping, reduce_lr]
    )
    
    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
    
    # Save final model
    model.save('final_densenet_finetuned_model.h5')
    
    return model, history

In [27]:
model, history = train_model()

Found 6166 images belonging to 2 classes.
Found 1761 images belonging to 2 classes.
Found 882 images belonging to 2 classes.
Number of classes detected: 2
Class indices: {'cats': 0, 'dogs': 1}
Epoch 1/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - accuracy: 0.6091 - loss: 1.1903 - val_accuracy: 0.9478 - val_loss: 0.6083 - learning_rate: 1.0000e-05
Epoch 2/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8452 - loss: 0.7385 - val_accuracy: 0.9727 - val_loss: 0.4929 - learning_rate: 1.0000e-05
Epoch 3/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 309s 2s/step - accuracy: 0.9068 - loss: 0.6094 - val_accuracy: 0.9807 - val_loss: 0.4481 - learning_rate: 1.0000e-05
Epoch 4/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - accuracy: 0.9338 - loss: 0.5395 - val_accuracy: 0.9835 - val_loss: 0.4176 - learning_rate: 1.0000e-05
Epoch 5/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.9402 - loss: 0.4992 - val_accuracy: 0.9847 - val_loss: 0.3973 - learning_rate: 1.0000e-05
28/28 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
[]